In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sportsball
from sportsball.nfl.nfl import Season, Team, Game
from sportsball.parsing import MasseyParser
from gspread_pandas import Spread, Client
from tqdm import tqdm_notebook
import pandas
from sportsball.nfl.nfl import GameRound
from collections import Counter, namedtuple

In [3]:
season_2018 = Season(
    year=2018,
    sheettitle='NFL 2018 Expected Wins',
)

In [4]:
season_2018.attach_sagarin_ratings()
season_2018.attach_538_ratings()
season_2018.attach_massey_ratings()
season_2018.attach_scorex_ratings()
season_2018.attach_vegas_ratings()

In [5]:
for game in season_2018.games:
    game.prob_win = game.predict_prob_scorex()

In [6]:
gm = season_2018.games[200]

In [9]:
gm

        | Week 14 | giants     at REDSKINS   | TIE

In [7]:
gm.home_team.ratings['vegas']

{1: {'rating': '0.000000'},
 2: {'rating': '0.000000'},
 3: {'rating': '-2.852558'},
 4: {'rating': '-2.301438'},
 5: {'rating': '-1.672499'},
 6: {'rating': '-2.937583'},
 7: {'rating': '-1.552182'},
 8: {'rating': '-2.151751'},
 9: {'rating': '-1.432074'},
 10: {'rating': '-0.040654'},
 11: {'rating': '-0.676500'},
 12: {'rating': '-1.157610'}}

In [8]:
season_2018.games[200].prob_win

0.7884498465850325

In [26]:
# season_2018.simulate()

In [10]:
value_map = {
    GameRound.REG.value: {
        1: 1.4,
        0: 0.9,
        -1: 0.4,
    },
    GameRound.WILDCARD.value: {
        1: 0,
        -1: 2.5,
    },
    GameRound.DIVISIONAL.value: {
        1: 5.5,
        -1: 5.5,
    },
    GameRound.CONFERENCE.value: {
        1: 4.0,
        -1: 4.0,
    },
    GameRound.SUPERBOWL.value: {
        1: 13.0,
        -1: 5.0,
    }
}

windict = {
    team: {
        GameRound.REG.value: {
            1: 0.0,
            0: 0.0,
            -1: 0.0,
        },
        GameRound.WILDCARD.value: {
            1: 0.0,
            -1: 0.0,
        },
        GameRound.DIVISIONAL.value: {
            1: 0.0,
            -1: 0.0,
        },
        GameRound.CONFERENCE.value: {
            1: 0.0,
            -1: 0.0,
        },
        GameRound.SUPERBOWL.value: {
            1: 0.0,
            -1: 0.0,
        },
    }
    for team in season_2018.teams.values()
}

team_values = {team: 0.0 for team in season_2018.teams.values()}
N = 5000.0

In [11]:
GameResult = namedtuple('GameResult', ['team', 'round', 'result'])

In [12]:
def season_gnrtr(N):
    for ii in tqdm_notebook(range(int(N))):
        season_2018.simulate()
        for game in season_2018.games + list(season_2018.playoff_games):
            yield GameResult(
                game.away_team.team_id,
                game.game_round,
                game.away_team.mapgmresult(game),
            )
            yield GameResult(
                game.home_team.team_id,
                game.game_round,
                game.home_team.mapgmresult(game),
            )

In [13]:
cntr = Counter(season_gnrtr(10000))

In [14]:
x = pandas.DataFrame(
    [(k.team, k.round.value, k.result, v / 10000.0) for k, v in cntr.items()],
    columns=[
        'team',
        'round',
        'result',
        'count',
    ])

In [15]:
xx = x.set_index(['team', 'result', 'round'])['count'].unstack(
    fill_value=0, level=[
        'round',
        'result',
    ])

In [16]:
dtf = pandas.DataFrame({
    'wins': xx[1][1],
    'ties': xx[1][0],
    'losses': xx[1][-1],
    'make_playoffs': xx[2][-1] + xx[3][-1] + xx[3][1],
    'wildcard': xx[2][1] + xx[2][-1],
    'first_round_bye': xx[3][-1] + xx[3][1] - xx[2][1],
    'divisional': xx[3][-1] + xx[3][1],
    'conference': xx[4][-1] + xx[4][1],
    'superbowl': xx[5][-1] + xx[5][1],
    'win_superbowl': xx[5][1],
})

In [17]:
dtf['value'] = (
    dtf.wins * 1.4 + dtf.ties * 0.9 + dtf.losses * 0.4 +
    dtf.make_playoffs * 2.5 + dtf.divisional * 3.0 + dtf.conference * 4.0 +
    dtf.superbowl * 5.0 + dtf.win_superbowl * 8.0)

In [18]:
dtf['wins'] = dtf.wins.round(1)
dtf['losses'] = dtf.losses.round(1)
dtf['make_playoffs'] = 100 * dtf.make_playoffs
dtf['wildcard'] = 100 * dtf.wildcard
dtf['first_round_bye'] = 100 * dtf.first_round_bye
dtf['divisional'] = 100 * dtf.divisional
dtf['conference'] = 100 * dtf.conference
dtf['superbowl'] = 100 * dtf.superbowl
dtf['win_superbowl'] = 100 * dtf.win_superbowl
dtf['value'] = dtf.value.round(2)

In [19]:
dtf.sort_values('value', ascending=False)[['value']]

,value
team,
saints,34.91
chiefs,33.86
rams,31.78
patriots,26.70
steelers,24.84
chargers,22.70
bears,21.26
texans,21.03
panthers,20.39


In [20]:
dtf.sort_values('value', ascending=False)

,wins,ties,losses,make_playoffs,wildcard,first_round_bye,divisional,conference,superbowl,win_superbowl,value
team,,,,,,,,,,,
saints,13.6,0.0,2.4,99.98,6.95,93.03,99.20,87.55,58.97,37.79,34.91
chiefs,13.7,0.0,2.3,100.00,13.04,86.96,96.05,80.07,56.72,28.88,33.86
rams,13.7,0.0,2.3,100.00,5.33,94.67,99.12,80.93,33.13,16.90,31.78
patriots,11.7,0.0,4.3,99.53,34.40,65.13,90.80,51.13,17.43,5.45,26.70
steelers,10.3,1.0,4.7,93.13,69.29,23.84,78.18,39.74,17.74,6.75,24.84
chargers,11.9,0.0,4.1,98.77,86.75,12.02,42.08,9.96,3.21,0.74,22.70
bears,10.4,0.0,5.6,87.38,80.07,7.31,56.04,10.52,2.67,0.82,21.26
texans,10.4,0.0,5.6,87.78,79.23,8.55,49.76,10.47,2.34,0.40,21.03
panthers,9.8,0.0,6.2,76.13,73.15,2.98,50.86,11.57,3.74,1.31,20.39


In [18]:
for team in season_2018.teams.values():
    team_id = team.team_id
    make_playoffs = ((cntr[GameResult(team_id, GameRound.DIVISIONAL, 1)] +
                      cntr[GameResult(team_id, GameRound.DIVISIONAL, -1)]) +
                     cntr[GameResult(team_id, GameRound.WILDCARD, -1)])
    wildcard = (cntr[GameResult(team_id, GameRound.WILDCARD, 1)] +
                cntr[GameResult(team_id, GameRound.WILDCARD, -1)])
    first_round_bye = (cntr[GameResult(team_id, GameRound.DIVISIONAL, 1)] +
                       cntr[GameResult(team_id, GameRound.DIVISIONAL, -1)] -
                       cntr[GameResult(team_id, GameRound.WILDCARD, 1)])
    divisional = (cntr[GameResult(team_id, GameRound.DIVISIONAL, 1)] +
                  cntr[GameResult(team_id, GameRound.DIVISIONAL, -1)])
    conference = (cntr[GameResult(team_id, GameRound.CONFERENCE, 1)] +
                  cntr[GameResult(team_id, GameRound.CONFERENCE, -1)])
    superbowl = (cntr[GameResult(team_id, GameRound.SUPERBOWL, 1)] +
                 cntr[GameResult(team_id, GameRound.SUPERBOWL, -1)])
    winsuperbowl = cntr[GameResult(team_id, GameRound.SUPERBOWL, 1)]
    ss = '''{:15} | {:5.1f} | {:4.0f} | {:6.1f} | {:6.1%} | {:6.1%} | {:6.1%} | {:6.1%} | {:6.1%} | {:6.1%} | {:6.1%} |'''.format(
        team.name,
        cntr[GameResult(team_id, GameRound.REG, 1)] / 5000.0,
        cntr[GameResult(team_id, GameRound.REG, 0)] / 5000.0,
        cntr[GameResult(team_id, GameRound.REG, -1)] / 5000.0,
        make_playoffs / 5000.0,
        wildcard / 5000.0,
        first_round_bye / 5000.0,
        divisional / 5000.0,
        conference / 5000.0,
        superbowl / 5000.0,
        winsuperbowl / 5000.0,
    )
    print(ss)

Buffalo         |   8.6 |    0 |   23.4 |   0.1% |   0.1% |   0.0% |   0.0% |   0.0% |   0.0% |   0.0% |
Miami           |  14.5 |    0 |   17.5 |  19.6% |  19.3% |   0.3% |   6.3% |   1.4% |   0.4% |   0.1% |
N.Y. Jets       |  12.4 |    0 |   19.6 |   5.6% |   5.4% |   0.1% |   1.8% |   0.3% |   0.1% |   0.1% |
New England     |  24.4 |    0 |    7.6 | 199.5% |  23.4% | 176.0% | 193.3% | 133.8% |  56.6% |  26.6% |
Baltimore       |  18.6 |    0 |   13.4 | 132.0% | 121.7% |  10.3% |  55.6% |  10.9% |   2.9% |   0.8% |
Cincinnati      |  18.1 |    0 |   13.9 | 108.8% | 104.5% |   4.3% |  60.7% |  17.6% |   7.0% |   3.1% |
Cleveland       |   9.4 |    2 |   20.6 |   1.1% |   1.1% |   0.0% |   0.4% |   0.1% |   0.0% |   0.0% |
Pittsburgh      |  18.0 |    2 |   12.0 | 134.7% | 117.0% |  17.7% |  89.1% |  33.4% |  14.6% |   6.5% |
Houston         |  18.7 |    0 |   13.3 | 150.3% | 142.4% |   7.9% |  91.9% |  23.2% |   7.6% |   2.5% |
Indianapolis    |  14.4 |    0 |   17.6 |  28.0% |  27.

In [138]:
# probability of making the playoffs
(cntr[GameResult('rams', GameRound.DIVISIONAL, 1)] + cntr[GameResult(
    'rams', GameRound.DIVISIONAL, -1)]) + cntr[GameResult(
        'rams', GameRound.WILDCARD, -1)]

979

In [139]:
# probability of wild card round
cntr[GameResult('rams', GameRound.WILDCARD, 1)] + cntr[GameResult(
    'rams', GameRound.WILDCARD, -1)]

188

In [144]:
# probability of divisional round
cntr[GameResult('rams', GameRound.DIVISIONAL, 1)] + cntr[GameResult(
    'rams', GameRound.DIVISIONAL, -1)]

918

In [145]:
# probability of first-round bye
cntr[GameResult('rams', GameRound.DIVISIONAL, 1)] + cntr[GameResult(
    'rams', GameRound.DIVISIONAL, -1)] - cntr[GameResult(
        'rams', GameRound.WILDCARD, 1)]

791

In [149]:
# probability of conference round
cntr[GameResult('rams', GameRound.CONFERENCE, 1)] + cntr[GameResult(
    'rams', GameRound.CONFERENCE, -1)]

609

In [150]:
# probability of super bowl
cntr[GameResult('rams', GameRound.SUPERBOWL, 1)] + cntr[GameResult(
    'rams', GameRound.SUPERBOWL, -1)]

295

In [151]:
# probability of winning super bowl
cntr[GameResult('rams', GameRound.SUPERBOWL, 1)]

158

In [131]:
print('{:5.2%}'.format(
    (cntr[GameResult('rams', GameRound.DIVISIONAL, 1)] + cntr[GameResult(
        'rams', GameRound.DIVISIONAL, -1)]) / 1000.))

91.80%


In [38]:
for ii in tqdm_notebook(range(int(N))):
    season_2018.simulate()
    for game in season_2018.games:
        away_team = game.away_team
        home_team = game.home_team
        team_values[away_team] += (
            value_map[game.game_round.value][away_team.mapgmresult(game)] / N)
        team_values[home_team] += (
            value_map[game.game_round.value][home_team.mapgmresult(game)] / N)
        windict[away_team][game.game_round.value][away_team.mapgmresult(
            game)] += (1 / N)
        windict[home_team][game.game_round.value][home_team.mapgmresult(
            game)] += (1 / N)
    for game in season_2018.playoff_games:
        away_team = game.away_team
        home_team = game.home_team
        team_values[away_team] += (
            value_map[game.game_round.value][away_team.mapgmresult(game)] / N)
        team_values[home_team] += (
            value_map[game.game_round.value][home_team.mapgmresult(game)] / N)
        windict[away_team][game.game_round.value][away_team.mapgmresult(
            game)] += (1 / N)
        windict[home_team][game.game_round.value][home_team.mapgmresult(
            game)] += (1 / N)

In [11]:
windict

{cardinals: Arizona: {1: {1: 4.1867999999999865,
   0: 0.0,
   -1: 11.813199999999146},
  2: {1: 0.0006000000000000001, -1: 0.0008},
  3: {1: 0.0, -1: 0.0006000000000000001},
  4: {1: 0.0, -1: 0.0},
  5: {1: 0.0, -1: 0.0}},
 falcons: Atlanta: {1: {1: 7.143000000006225, 0: 0.0, -1: 8.857000000006035},
  2: {1: 0.05639999999999977, -1: 0.08640000000000042},
  3: {1: 0.021999999999999978, -1: 0.04519999999999984},
  4: {1: 0.009199999999999998, -1: 0.012800000000000008},
  5: {1: 0.0053999999999999986, -1: 0.0038000000000000013}},
 ravens: Baltimore: {1: {1: 8.029200000007965, 0: 0.0, -1: 7.970800000007972},
  2: {1: 0.11420000000000122, -1: 0.1350000000000018},
  3: {1: 0.06159999999999974, -1: 0.10200000000000087},
  4: {1: 0.022199999999999977, -1: 0.03939999999999987},
  5: {1: 0.007399999999999995, -1: 0.014800000000000013}},
 bills: Buffalo: {1: {1: 6.111400000004048, 0: 0.0, -1: 9.888600000003631},
  2: {1: 0.018799999999999997, -1: 0.022599999999999974},
  3: {1: 0.005599999999999

In [40]:
print('''{:15} | {:5s} | {:4s} | {:5s} | {:6s} |'''.format(
    'Team Name', 'Wins', 'Ties', 'Losses', 'Plyffs'))
print('-' * 41)
for team, entry in windict.items():
    ss = '''{:15} | {:5.1f} | {:4d} | {:6.1f} | {:6.1%} | {:6.1%} |'''.format(
        team.name,
        entry[1][1],
        int(entry[1][0]),
        entry[1][-1],
        entry[2][1] + entry[2][-1],
        entry[3][1] + entry[3][-1],
    )
    print(ss)

#     ss = '''{:4}| {:10}| {:15}| {:2} - {:2} - {:2} | {:>4}'''.format(
#         team.name
#         '{:.0%}'.format(self.winpct),
#     )

Team Name       | Wins  | Ties | Losses | Plyffs |
-----------------------------------------
Arizona         |   4.6 |    0 |   11.4 |   0.4% |   0.1% |
Atlanta         |   7.4 |    0 |    8.6 |  20.7% |  12.7% |
Baltimore       |   9.6 |    0 |    6.4 |  44.7% |  39.3% |
Buffalo         |   5.0 |    0 |   11.0 |   0.8% |   0.3% |
Carolina        |   8.4 |    0 |    7.6 |  33.4% |  24.5% |
Chicago         |   9.0 |    0 |    7.0 |  43.5% |  35.4% |
Cincinnati      |   8.9 |    0 |    7.1 |  37.3% |  30.7% |
Cleveland       |   6.2 |    0 |    8.8 |   6.5% |   2.9% |
Dallas          |   7.7 |    0 |    8.3 |  28.6% |  16.6% |
Denver          |   6.8 |    0 |    9.2 |   7.3% |   2.7% |
Detroit         |   6.8 |    0 |    9.2 |  12.2% |   7.4% |
Green Bay       |   7.6 |    0 |    7.4 |  31.0% |  20.0% |
Houston         |   8.3 |    0 |    7.7 |  33.1% |  22.7% |
Indianapolis    |   5.7 |    0 |   10.3 |   2.5% |   1.4% |
Jacksonville    |   9.2 |    0 |    6.8 |  43.4% |  39.9% |
Kansas 

In [39]:
for k, v in sorted(team_values.items(), key=lambda x: x[1], reverse=True):
    print('{:15} | ${:0.2f}'.format(k.name, v))

L.A. Rams       | $31.45
Kansas City     | $27.87
New England     | $25.43
New Orleans     | $23.24
Baltimore       | $19.87
Minnesota       | $19.71
L.A. Chargers   | $19.61
Jacksonville    | $19.31
Chicago         | $18.88
Cincinnati      | $18.62
Pittsburgh      | $18.60
Philadelphia    | $18.37
Carolina        | $17.38
Miami           | $17.10
Houston         | $16.95
Green Bay       | $16.58
Washington      | $16.15
Tampa Bay       | $15.85
Seattle         | $15.84
Dallas          | $15.71
Atlanta         | $15.15
Tennessee       | $15.10
N.Y. Jets       | $14.68
Detroit         | $13.97
Denver          | $13.53
Cleveland       | $13.42
Indianapolis    | $12.19
N.Y. Giants     | $12.19
Oakland         | $11.85
San Francisco   | $11.70
Buffalo         | $11.48
Arizona         | $11.01


In [148]:
sum([v / 4000 for k, v in team_values.items()])

548.8000000002513

In [16]:
_team_values = {team: 0.0 for team in season_2018.teams.values()}
for ii in range(2000):
    season_2018.simulate()
    for game in season_2018.games:
        away_team = game.away_team
        home_team = game.home_team
        _team_values[away_team] += value_map[game.game_round.value][
            away_team._mapgmresult(game)]
        _team_values[home_team] += value_map[game.game_round.value][
            home_team._mapgmresult(game)]
    for game in season_2018.playoff_games:
        away_team = game.away_team
        home_team = game.home_team
        _team_values[away_team] += value_map[game.game_round.value][
            away_team._mapgmresult(game)]
        _team_values[home_team] += value_map[game.game_round.value][
            home_team._mapgmresult(game)]

In [22]:
for k, v in sorted(_team_values.items(), key=lambda x: x[1], reverse=True):
    print('{:15} | ${:0.2f}'.format(k.name, v / 2000.0))

L.A. Rams       | $24.10
Kansas City     | $21.59
Jacksonville    | $19.93
Baltimore       | $19.49
Philadelphia    | $19.31
New Orleans     | $18.59
New England     | $18.51
Miami           | $18.19
L.A. Chargers   | $18.11
Chicago         | $17.69
Tampa Bay       | $17.60
Carolina        | $17.57
Cincinnati      | $17.32
Atlanta         | $17.02
Tennessee       | $16.86
Minnesota       | $16.83
Pittsburgh      | $16.29
Washington      | $15.90
Detroit         | $15.85
Seattle         | $15.73
Dallas          | $15.31
San Francisco   | $15.18
Cleveland       | $15.08
Indianapolis    | $14.98
Buffalo         | $14.98
Denver          | $14.76
N.Y. Jets       | $14.70
Green Bay       | $14.65
N.Y. Giants     | $13.81
Houston         | $13.23
Oakland         | $12.57
Arizona         | $12.05


In [20]:
sum([v / 2000 for k, v in _team_values.items()])

533.8000000003052

In [74]:
games = season_2018.sim_playoffs()

In [76]:
game = games.pop()

In [67]:
team_to_seed = {
    team: seed
    for cnfrnce_str, cnfrnce in playoff_seeds.items()
    for seed, team in cnfrnce.items()
}

In [69]:
afc_seeds, nfc_seeds = playoff_seeds.values()

In [54]:
def sim_playoffs(afc_seeds, nfc_seeds, season):
    # Round 1
    # afc
    # 5 vs. 4 (home)
    game1 = Game(
        week=18,
        season=season,
        neutral=False,
        playoff=True,
        away_team=afc_seeds[5],
        home_team=afc_seeds[4],
        away_score=None,
        home_score=None)

    # 6 vs. 3 (home)
    game2 = Game(
        week=18,
        season=season,
        neutral=False,
        playoff=True,
        away_team=afc_seeds[6],
        home_team=afc_seeds[3],
        away_score=None,
        home_score=None)

    # nfc
    # 5 vs. 4 (home)
    game3 = Game(
        week=18,
        season=season,
        neutral=False,
        playoff=True,
        away_team=nfc_seeds[5],
        home_team=nfc_seeds[4],
        away_score=None,
        home_score=None)

    # 6 vs. 3 (home)
    game4 = Game(
        week=18,
        season=season,
        neutral=False,
        playoff=True,
        away_team=nfc_seeds[6],
        home_team=nfc_seeds[3],
        away_score=None,
        home_score=None)

    games = {
        game1,
        game2,
        game3,
        game4,
    }

    for game in games:
        home_adv = season_2018.get_home_adv(system='vegas', week=18)
        game.prob_win = game.predict_prob_vegas(0.1354771, home_adv=home_adv)
        game.simulate()

    winners = {game.winner() for game in games}

    afc_hiseed, afc_lowseed = sorted(
        {k
         for k, v in afc_seeds.items() if v in winners})
    nfc_hiseed, nfc_lowseed = sorted(
        {k
         for k, v in nfc_seeds.items() if v in winners})

    # Divisional Round
    # afc
    # lowest vs. 1 (home)
    game5 = Game(
        week=19,
        season=season,
        neutral=False,
        playoff=True,
        away_team=afc_seeds[afc_lowseed],
        home_team=afc_seeds[1],
        away_score=None,
        home_score=None)

    # 6 vs. 3 (home)
    game6 = Game(
        week=19,
        season=season,
        neutral=False,
        playoff=True,
        away_team=afc_seeds[afc_hiseed],
        home_team=afc_seeds[2],
        away_score=None,
        home_score=None)

    # nfc
    # 5 vs. 4 (home)
    game7 = Game(
        week=19,
        season=season,
        neutral=False,
        playoff=True,
        away_team=nfc_seeds[nfc_lowseed],
        home_team=nfc_seeds[1],
        away_score=None,
        home_score=None)

    # 6 vs. 3 (home)
    game8 = Game(
        week=19,
        season=season,
        neutral=False,
        playoff=True,
        away_team=nfc_seeds[nfc_hiseed],
        home_team=nfc_seeds[2],
        away_score=None,
        home_score=None)

    games = {
        game5,
        game6,
        game7,
        game8,
    }

    for game in games:
        home_adv = season_2018.get_home_adv(system='vegas', week=19)
        game.prob_win = game.predict_prob_vegas(0.1354771, home_adv=home_adv)
        game.simulate()

    # Conference Championships
    game9 = Game(
        week=20,
        season=season,
        neutral=True,
        playoff=True,
        away_team=game5.winner(),
        home_team=game6.winner(),
        away_score=None,
        home_score=None)

    game10 = Game(
        week=20,
        season=season,
        neutral=True,
        playoff=True,
        away_team=game7.winner(),
        home_team=game8.winner(),
        away_score=None,
        home_score=None)

    games = {
        game9,
        game10,
    }

    for game in games:
        home_adv = season_2018.get_home_adv(system='vegas', week=19)
        game.prob_win = game.predict_prob_vegas(0.1354771, home_adv=home_adv)
        game.simulate()

    # Superbowl
    game11 = Game(
        week=22,
        season=season,
        neutral=True,
        playoff=True,
        away_team=game9.winner(),
        home_team=game10.winner(),
        away_score=None,
        home_score=None)

    home_adv = season_2018.get_home_adv(system='vegas', week=19)
    game11.prob_win = game11.predict_prob_vegas(0.1354771, home_adv=home_adv)
    game11.simulate()

    return {
        game1, game2, game3, game4, game5, game6, game7, game8, game9, game10,
        game11
    }

In [ ]:
for game in (game1:
    away_team = game.away_team
    home_team = game.home_team
    away_team.games.append(game)
    home_team.games.append(game)

In [41]:
teammap['AFC']['cnfrnce_seeds']

{1: patriots: New England,
 2: chargers: L.A. Chargers,
 3: steelers: Pittsburgh,
 4: texans: Houston,
 5: dolphins: Miami,
 6: broncos: Denver}

In [62]:
winners = sim_playoffs(
    teammap['AFC']['cnfrnce_seeds'],
    teammap['NFC']['cnfrnce_seeds'],
    season=2018)

In [63]:
winners

{        | Week 18 | broncos    at STEELERS   | steelers,
         | Week 18 | dolphins   at TEXANS     | texans,
         | Week 18 | falcons    at REDSKINS   | redskins,
         | Week 18 | saints     at VIKINGS    | vikings,
         | Week 19 | redskins   at RAMS       | redskins,
         | Week 19 | steelers   at CHARGERS   | chargers,
         | Week 19 | texans     at PATRIOTS   | patriots,
         | Week 19 | vikings    at BUCCANEERS | vikings,
         | Week 20 | patriots   vs chargers   | patriots,
         | Week 20 | redskins   vs vikings    | vikings,
         | Week 22 | patriots   vs vikings    | patriots}

In [53]:
{winner.division for winner in winners}

{'AFC East', 'AFC North', 'NFC East', 'NFC North'}

In [32]:
for game in sim_playoffs(
        teammap['AFC']['cnfrnce_seeds'],
        teammap['NFC']['cnfrnce_seeds'],
        season=2018):
    home_adv = season_2018.get_home_adv(system='vegas', week=18)
    game.prob_win = game.predict_prob_vegas(0.1354771, home_adv=home_adv)
    game.simulate()
    print(game)

        | Week 18 | dolphins   at TEXANS     | texans
        | Week 18 | broncos    at STEELERS   | broncos
        | Week 18 | falcons    at REDSKINS   | falcons
        | Week 18 | saints     at VIKINGS    | vikings


In [30]:
import functools

In [248]:
@functools.total_ordering
class MyNum(int):
            
    def __lt__(self, other, switch=True):
        if switch:
            return int(self) > int(other)
        return int(self) < int(other)

In [256]:
numA = MyNum(3)
numB = MyNum(3)
numC = MyNum(5)

In [258]:
numA is numB

False

In [147]:
def quick_sort(items, switch=None):
    """ Implementation of quick sort """
    if len(items) > 1:
        pivot_index = int(len(items) / 2)
        smaller_items = []
        larger_items = []

        for i, val in enumerate(items):
            if i != pivot_index:
                if val.__lt__(items[pivot_index], switch=switch):
                    smaller_items.append(val)
                else:
                    larger_items.append(val)

        return quick_sort(
            smaller_items, switch=switch) + [items[pivot_index]] + quick_sort(
                larger_items, switch=switch)
    else:
        return items

In [179]:
for cnfrnce_str, cnfrnce in season_2018.league.items():
    print(cnfrnce_str)
    for dvsn_str, dvsn in cnfrnce.items():
        print('  {}'.format(dvsn_str))
        for team in sorted(dvsn, reverse=True):
            print('    {}'.format('{:15}| {:2} - {:2} - {:2} | {:>4}'.format(
                team.name,
                team.wins,
                team.ties,
                team.losses,
                '{:.2%}'.format(team.winpct),
            )))

AFC
  AFC East
    New England    | 12 -  0 -  4 | 75.00%
    N.Y. Jets      |  8 -  0 -  8 | 50.00%
    Miami          |  7 -  0 -  9 | 43.75%
    Buffalo        |  6 -  0 - 10 | 37.50%
  AFC North
    Baltimore      | 12 -  0 -  4 | 75.00%
    Pittsburgh     |  8 -  1 -  7 | 53.12%
    Cincinnati     |  8 -  0 -  8 | 50.00%
    Cleveland      |  6 -  1 -  9 | 40.62%
  AFC West
    Kansas City    | 10 -  0 -  6 | 62.50%
    Denver         |  9 -  0 -  7 | 56.25%
    L.A. Chargers  |  8 -  0 -  8 | 50.00%
    Oakland        |  4 -  0 - 12 | 25.00%
  AFC South
    Jacksonville   |  9 -  0 -  7 | 56.25%
    Tennessee      |  7 -  0 -  9 | 43.75%
    Indianapolis   |  7 -  0 -  9 | 43.75%
    Houston        |  7 -  0 -  9 | 43.75%
NFC
  NFC East
    Philadelphia   |  9 -  0 -  7 | 56.25%
    N.Y. Giants    |  9 -  0 -  7 | 56.25%
    Dallas         |  7 -  0 -  9 | 43.75%
    Washington     |  6 -  0 - 10 | 37.50%
  NFC North
    Minnesota      |  9 -  1 -  6 | 59.38%
    Chicago        |

In [133]:
dvsn = season_2018.league['NFC']['NFC North']

In [129]:
def division_winner(dvsn):
    bestrecord = max([team.wins + 0.5 * team.ties for team in dvsn])

    division_champs = [
        team for team in dvsn if team.wins + 0.5 * team.ties == bestrecord
    ]
    if len(division_champs) == 1:
        return division_champs[0]
    return divisional_tiebreak(division_champs)


def divisional_tiebreak(teams):
    if len(teams) == 1:
        return teams[0]
    _teams = head_to_head(teams)
    if len(_teams) == 1:
        print('tie broken via head-to-head record')
        return _teams[0]
    nteams = len(_teams)
    _teams = division_records(_teams)
    if len(_teams) == 1:
        print('tie broken via divisional record')
        return _teams[0]
    if len(_teams) != nteams:
        return divisional_tiebreak(_teams)
    # now just return random team
    print('tie broken via coin toss')
    return numpy.random.choice(_teams, size=1)[0]


def head_to_head(teams):
    cntnr = {
        team: wins(
            Counter([
                team._mapgmresult(game) for game in team.games
                if game.away_team in teams and game.home_team in teams
            ]))
        for team in teams
    }
    bestrecord = max(list(cntnr.values()))
    return [
        team for team, wins in cntnr.items()
        if wins == max(list(cntnr.values()))
    ]


def wins(counter):
    return float(counter[1] + 0.5 * counter[0])


def division_records(teams):
    cntnr = {
        team: wins(
            Counter([
                team._mapgmresult(game) for game in team.games
                if game.away_team.division == game.home_team.division
            ]))
        for team in teams
    }
    bestrecord = max(list(cntnr.values()))
    return [
        team for team, wins in cntnr.items()
        if wins == max(list(cntnr.values()))
    ]

In [149]:
def conference_wildcard(conference):
    bestrecord = max([team.wins + 0.5 * team.ties for team in conference])

    conference_champs = [
        team for team in conference
        if team.wins + 0.5 * team.ties == bestrecord
    ]
    if len(conference_champs) == 1:
        return conference_champs[0]
    return conference_champs

In [ ]:
division_winner(dvsn)

In [157]:
division_winners = {
    cnfrnce_str:
    {dvsn_str: division_winner(dvsn)
     for dvsn_str, dvsn in cnfrnce.items()}
    for cnfrnce_str, cnfrnce in season_2018.league.items()
}

In [163]:
sorted(list(division_winners['NFC'].values()))

[eagles: Philadelphia, packers: Green Bay, panthers: Carolina, rams: L.A. Rams]

In [161]:
for team in division_winners['NFC'].values():
    print('    {}'.format('{:15}| {:2} - {:2} - {:2} | {:>4}'.format(
        team.name,
        team.wins,
        team.ties,
        team.losses,
        '{:.2%}'.format(team.winpct),
    )))

    Philadelphia   | 11 -  0 -  5 | 68.75%
    Green Bay      | 12 -  1 -  3 | 78.12%
    Carolina       | 12 -  0 -  4 | 75.00%
    L.A. Rams      | 13 -  0 -  3 | 81.25%


In [146]:
teams_left = [team for team in season_2018.teams.values() if team not in division_winners]

In [152]:
conference = season_2018.league['NFC']

In [154]:
conference_wildcard([team for team in teams_left if team.conference == 'AFC'])

chargers: L.A. Chargers

In [139]:
for cnfrnce_str, cnfrnce in season_2018.league.items():
    print(cnfrnce_str)
    for dvsn_str, dvsn in cnfrnce.items():
        print('  {}'.format(dvsn_str))

        team = division_winner(dvsn)
        print('    {}'.format('{:15}| {:2} - {:2} - {:2} | {:>4}'.format(
            team.name,
            team.wins,
            team.ties,
            team.losses,
            '{:.2%}'.format(team.winpct),
        )))

AFC
  AFC East
    New England    |  9 -  0 -  7 | 56.25%
  AFC North
    Baltimore      | 12 -  0 -  4 | 75.00%
  AFC West
    Kansas City    | 12 -  0 -  4 | 75.00%
  AFC South
    Indianapolis   | 11 -  0 -  5 | 68.75%
NFC
  NFC East
    Philadelphia   | 11 -  0 -  5 | 68.75%
  NFC North
    Green Bay      | 12 -  1 -  3 | 78.12%
  NFC South
    Carolina       | 12 -  0 -  4 | 75.00%
  NFC West
    L.A. Rams      | 13 -  0 -  3 | 81.25%


In [14]:
season_2018.attach_sagarin_ratings()
season_2018.attach_538_ratings()
season_2018.attach_massey_ratings()
season_2018.attach_scorex_ratings()
season_2018.attach_vegas_ratings()

In [15]:
season_2018.set_game_probs('vegas')

In [16]:
season_2018.simulate()

In [344]:
season_2018.teams['Miami'].losses

8

In [308]:
df['pct'] = df.wins / (df.wins + df.losses)

In [312]:
df.sort_values(
    ['division', 'pct'], ascending=[True, False])[[
        'name',
        'conference',
        'division',
        'wins',
        'losses',
        'pct',
    ]]

,name,conference,division,wins,losses,pct
1,Miami,AFC,AFC East,10,6,0.6250
2,N.Y. Jets,AFC,AFC East,10,6,0.6250
3,New England,AFC,AFC East,10,6,0.6250
0,Buffalo,AFC,AFC East,1,15,0.0625
4,Baltimore,AFC,AFC North,11,5,0.6875
5,Cincinnati,AFC,AFC North,8,8,0.5000
7,Pittsburgh,AFC,AFC North,7,9,0.4375
6,Cleveland,AFC,AFC North,6,10,0.3750
11,Jacksonville,AFC,AFC South,13,3,0.8125
15,Tennessee,AFC,AFC South,9,7,0.5625


In [1]:
import pandas
import sklearn

from sklearn_pandas import DataFrameMapper, cross_val_score
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer, log_loss

In [168]:
from sklearn.model_selection import GroupKFold

In [148]:
from sklearn.linear_model import SGDClassifier, LogisticRegression

In [8]:
dtf = pandas.read_csv('./../data/processed/nfl_data.csv')
dtf = dtf.loc[(dtf.home_win == 0) | (dtf.home_win == 1)]

In [239]:
dtf

,season,five38_home_rating,five38_away_rating,massey_home_rating,massey_away_rating,sagarin_home_rating,sagarin_away_rating,home_win,gamenum,home_adv
0,2017,1434,1559,24.83,22.80,17.12,20.50,1.0,1,1
1,2017,1529,1600,21.28,18.56,23.01,21.80,1.0,2,1
2,2017,1476,1497,21.00,24.19,20.23,18.40,1.0,3,1
3,2017,1498,1480,21.69,11.05,24.08,17.84,1.0,4,1
4,2017,1623,1489,32.83,15.50,27.78,19.01,1.0,5,1
5,2017,1489,1387,29.52,23.75,24.02,11.43,0.0,6,1
6,2017,1455,1514,17.59,25.61,19.77,21.57,0.0,7,1
7,2017,1598,1436,31.03,18.36,27.66,19.74,1.0,8,1
8,2017,1282,1460,20.29,22.22,12.33,14.98,0.0,9,1
9,2017,1608,1477,24.66,21.94,26.23,20.71,0.0,10,1


In [144]:
pipe = sklearn.pipeline.Pipeline([
    ('featurize',
     DataFrameMapper(
         [
             (['five38_home_rating'], StandardScaler()),
             (['five38_away_rating'], StandardScaler()),
             (['massey_home_rating'], StandardScaler()),
             (['massey_away_rating'], StandardScaler()),
             (['sagarin_home_rating'], StandardScaler()),
             (['sagarin_away_rating'], StandardScaler()),
         ],
         sparse=True)),
    ('lm',
     SGDClassifier(
         alpha=0.08,
         loss='log',
         fit_intercept=False,
         penalty='l2',
         max_iter=1000,
         tol=1e-3,
     )),
])

In [230]:
pipe = sklearn.pipeline.Pipeline([
    ('featurize',
     DataFrameMapper(
         [
             ([
                 'five38_away_rating',
                 'five38_home_rating',
                 'massey_away_rating',
                 'massey_home_rating',
                 'sagarin_away_rating',
                 'sagarin_home_rating',
             ], StandardScaler()),
         ],
         sparse=True)),
    ('lm', LogisticRegression(C=0.4)),
])

In [232]:
x = pipe.named_steps['featurize']

In [236]:
x.features[0][1].scale_

array([97.86976773, 97.86976773,  4.51784368,  4.51784368,  4.5378788 ,
        4.5378788 ])

In [237]:
x.features[0][1].mean_

array([1513.71610169, 1513.71610169,   23.00144068,   23.00144068,
         21.25025424,   21.25025424])

In [ ]:
cv = GroupKFold(n_splits=20)

In [206]:
xxx = cross_val_score(
    pipe,
    X=dtf,
    y=dtf.home_win,
    groups=dtf.gamenum,
    scoring=make_scorer(
        log_loss,
        greater_is_better=False,
        needs_proba=True,
    ),
    cv=cv.split(X=dtf, groups=dtf.gamenum),
    n_jobs=-1,
)

In [207]:
xxx.mean()

-0.5844360835663382

In [ ]:
clf = GridSearchCV(svc, parameters, scor)

In [231]:
pipe.fit(X=dtf, y=dtf.home_win)

Pipeline(memory=None,
     steps=[('featurize', DataFrameMapper(default=False, df_out=False,
        features=[(['five38_away_rating', 'five38_home_rating', 'massey_away_rating', 'massey_home_rating', 'sagarin_away_rating', 'sagarin_home_rating'], StandardScaler(copy=True, with_mean=True, with_std=True))],
        input_df=Fa...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [220]:
lm = pipe.named_steps['lm']

In [235]:
lm.coef_

array([[-0.48811602,  0.48811602,  0.83260993, -0.83260993,  0.17701139,
        -0.17701139]])

In [238]:
0.48811602/97.86976773

0.004987403478330498